In [88]:
import array
import random

# leggo file

In [89]:
File = open("f_libraries_of_the_world.txt","r")
Lines = File.readlines()

<h1>Lettura della istanza</h1>
<p> Il primo rigo contiene le informazioni relative al numero di libri B, numero di librerie L e numero di giorni D.
La riga successiva contiene lo score associato ad ogni libro, utilizziamo un array chiamato Books_score così definito:
Books_score -> chiave = libro, valore = score.

Successivamente ci sono solo righe dedicate alle librerie, la prima riga contiene come informazioni il numero di libri nella libreria, la durata del periodo di signup e il numero di libri scannerizzati ogni giorno.
<br>Il rigo successivo contiene tutti gli ID dei libri in possesso a quella liberia. In questo caso abbiamo utilizzato un array multidimensionale così definito:

<ol> <li>Library -> chiave = lib_id, valore = (num_of_books, signup_days, num_of_books_per_day, books) dove:
        <ul> <li>num_of_books - numero di libri presenti nella libreria,</li>
             <li>signup_days  - numero di giorni necessari per il signup della libreria, </li>
             <li>num_of_books_per_day - numero massimo di libri che possono essere fotocopiati in un giorno nella libreria,</li>
             <li>books - lista di libri presenti nella libreria. </li>
        </ul> </li>
</ol>

In [90]:
Split_Line = Lines[0].split(" ")

Num_Of_Books = int(Split_Line[0])
Num_Of_Libraries = int(Split_Line[1])
Num_Of_Days = int(Split_Line[2])

Books_score = {}
Split_Line = Lines[1].split(" ")
for i in range(Num_Of_Books) :
    Books_score[i] = int(Split_Line[i])

Library = {}

Line_number = 2
for j in range(Num_Of_Libraries) :
    Split_Line = Lines[Line_number].split(" ")

    Library[j] = (int(Split_Line[0]), int(Split_Line[1]), int(Split_Line[2]), [] )

    Split_Line = Lines[Line_number + 1].split(" ")
    for p in Split_Line:
        Library[j][3].append(int(p))

    #ordino i libri per score crescente
    Library[j][3].sort(key = lambda x: Books_score[x], reverse=True)
    Line_number = Line_number + 2

size = Num_Of_Libraries

<h1> Rappresentazione soluzione </h1>
La soluzione consiste nel ordine in cui veniene eseguito il signup delle varie librerie, quindi la soluzione è rappresentata da un vettore
di numeri unici che vanno da 0 a size-1, cioè il numero di librerie
lette nel file.  

<h1> Funzione obbiettivo </h1>
<p>  Prende in input uno stato, l'array che rappresenta una soluzione del problema, per ogni libreria:
     <ol>
         <li> Tramite la variabile Current day si tiene conto del periodo di sign up e se la libreria può iniziare a lavorare.
        </li>
         <li> Si calcola quanti libri possono essere scannerizzati fino alla fine dei D giorni </li>
         <li> Un ciclo For itera tutti i libri della lista, tenendo conto di quelli già scannerizzati e del numero
              massimo di libri che si possono scannerizzare.
     </ol>

In [91]:
def get_cost(state):
    Current_day = 0
    Number_of_Scanned_Books = 0
    Total_Score = 0
    Total_Copied_Book = 0
    Scanned_Book = {} # cliave libro, valore libreria

    for lib_id in state:
        Current_day += Library[lib_id][1]
        if Current_day >= Num_Of_Days:
            break

        Max_Num_Book = (Num_Of_Days - Current_day - 1) * Library[lib_id][2]
        Max_Num_Book = min(Max_Num_Book, Library[lib_id][0])
        Copied_Book  = 0

        for book_id in Library[lib_id][3]:
            #se non ho ancora scannerizzato il libro
            if (book_id not in Scanned_Book):
                #salvo il libro scannerizzato
                Scanned_Book[book_id] = lib_id
                #aggiorno punteggio
                Total_Score += Books_score[book_id]

                Copied_Book       += 1
                Total_Copied_Book += 1
                if Copied_Book > Max_Num_Book:
                    break
    return Total_Score

<h1>
    Definizione Mossa
</h1>
La mossa è definita da due operazioni consegutive:

1.   Uno scambio tra due elementi casuali
2.   Uno shuffle casuale di un segmento della soluzione.

La prima operazione avviene sempre e il risultato farà parte del intorno, invece la seconda operazione ha una probabilità di essere
eseguita che dipende da quante iterazioni sono state svolte,
in particolare: la probabilità è pari a $\frac{\sqrt{k}}{100}$, quindi
dopo 100 iterazioni c'è una probabilità del 10% di eseguire uno shuffle.

In questo modo più si va avanti nel algoritmo e più è probabile l'effetto della seconda operazione, rendendo l'intorno più grande
e fornendo candidati più diversificati.  

Da sottolineare che l'intorno ha in ogni caso, se avviene sempre uno shuffle, una dimensione massima pari
a 2 volte il valore: $10\sqrt{size}$.

In [92]:
import math

def get_neighboorhood_solutions(state:list, k) -> list:
        fin = int(math.sqrt(size)*10)
        for _ in range(fin):
          new = state.copy()
          i2 = random.randint(0, size-1)
          i1 = random.randint(0, size-1)
          new[i1], new[i2] = new[i2], new[i1]
          yield new

          if random.random() < (math.sqrt(k)/100):
            s1 = random.randint(0,size-1)
            s2 = random.randint(s1,size-1)
            shuf = new[s1:s2]
            random.shuffle(shuf)
            new[s1:s2] = shuf
            yield new


<h1>Algoritmo di Tabu Search</h1>
L'algoritmo è un implementazione abbastanza semplice e standard:
  
1. Il criterio d'arresto è definito da un numero prefissato di iterazioni;
2. Il candidato è scelto con logica best improvment, quindi il migliore
nell'intorno.
3. Come meccanismo di l'intensification è stato introdotto un restart,
tale avviene ogni 10 iterazioni senza miglioramenti.
4. La tabu list ha dimensione fissa e una memoria basata su hashtable

In [93]:
def run_tabu_search(initial_solution, iterations: int, tabu_size: int, verbose) -> list:
        best_solution      = initial_solution
        best_solution_cost = get_cost(initial_solution)

        best_candidate      = initial_solution
        best_candidate_cost = get_cost(initial_solution)

        tabu_list = [initial_solution]
        worst_case = 0

        for i in range(iterations):
            best_candidate = max(get_neighboorhood_solutions(best_candidate, i+1), key = lambda x: tabu_check(x, tabu_list))
            best_candidate_cost = get_cost(best_candidate)

            if best_candidate_cost > best_solution_cost:
                best_solution      = best_candidate
                best_solution_cost = best_candidate_cost
            else:
                worst_case += 1
                if worst_case > 10:
                  best_candidate = best_solution
                  best_candidate_cost =  best_solution_cost
                  worst_case = 0

            tabu_list.append(best_candidate)
            if len(tabu_list) > tabu_size:
                tabu_list.pop(0)

            if verbose == True:
                print(i, best_solution_cost)

        return best_solution_cost

def tabu_check(x, tabu_list):
    if x not in tabu_list:
      return get_cost(x)
    return 0

<h1> Istanza di innesco </h1>
La soluzione iniziale di innesco è una ottenuta utilizzando un approccio greedy, infatti lo scheduling viene ottenuto ordinando le libreria secondo un fattore così calcolato come $\frac{Total\_Score}{Signup\_days}$, dove:

2. Signup\_days sono appunti i giorni necessari alla libreria i per iniziare a scannerizzare libri
3. Total\_Score è ottenuto sommando lo score dei libri per una quantità pari a: $(Num\_of\_days - Signup\_Days-1)*Books\_for\_days)$

In pratica è un approssimazione che non tiene conto del vincolo
di dover scannerizzare una e una sola volta i libri, in quasi tutti
i dataset fornisce un buon punto di partenza per l'algoritmo.

In [94]:
def get_estimate(lib_id):
  Total_Score = 0

  Max_Num_Book = (Num_Of_Days - Library[lib_id][1] - 1) * Library[lib_id][2]
  Bookcount    = min(Max_Num_Book, Library[lib_id][0])

  scanned_book = Library[lib_id][3][0:Bookcount]
  for book_id in scanned_book:
    Total_Score  += Books_score[book_id]

  return Total_Score/Library[lib_id][1]


G_start = [v for v in Library.keys()]
G_start.sort(key = lambda x: get_estimate(x), reverse=True)

print(get_cost(G_start))

5236238


In [ ]:
for _ in range(1):
    a = run_tabu_search(G_start, 200, 50, True)
    print("--- best solution: ---", a)

<table><thead><tr><th>Istanza</th><th>Risultato Tabu</th><th>Tempo di calcolo </th><th>Risultato Genetico</th><th>Tempo di calcolo</th><th>Risultato concorrenti 530th</th><th>Tempo di calcolo</th></tr></thead><tbody><tr><td>A - Example</td><td>21</td><td>0.1 sec</td><td>21</td><td>0.02 sec</td><td>21</td><td>0.003 sec</td></tr><tr><td rowspan="3">B - Read On</td><td>20 iter - 5.822.900</td><td>5 sec </td><td rowspan="3">5.893.800<br><br>popsize = 150<br>ngen   = 200<br>cxpb   = 0.3<br>mutpb = 0.7<br>tool_m = 0.04</td><td rowspan="3">1 min per arrivare<br>intorno a <br>5.800.000, <br>convergenza a<br>circa 10 min</td><td rowspan="3">5.822.900</td><td rowspan="15">Tempo <br>che varia dai <br>15 ai 20 minuti <br>per ogni <br>dataset.</td></tr><tr><td>100 iter - 5.822.900</td><td>18 sec </td></tr><tr><td>200 iter - 5.822.900</td><td>35 sec </td></tr><tr><td rowspan="3">C - incunabula</td><td>20 iter - 5.648.064</td><td>7 min </td><td rowspan="3">1.737.140<br>popsize = 100<br>ngen    = 200<br>cxpb    = 0.5<br>mutpb  = 0.7<br>tool_m = 0.02</td><td rowspan="3">in un 1 min <br>intorno al <br>1.300.000, <br>convergenza in <br>circa 10 min</td><td rowspan="3">5.690.378</td></tr><tr><td>100 iter - 5.652.179</td><td>40 min</td></tr><tr><td>200 iter - 5.652.830</td><td>76 min</td></tr><tr><td rowspan="3">D - Tough choices</td><td>20 iter - 4.823.260</td><td>38 min </td><td rowspan="3">4.415.000<br>pop_size = 100<br>ngen = 50<br>cxpb = 0.5<br>mutpb = 0.7<br>tool_m = 0.02</td><td rowspan="3">1 min per <br>arrivare intorno a<br>4.300.000, <br>converge in <br>circa 10 min</td><td rowspan="3">5.028.530</td></tr><tr><td>60 iter - 4.831.775</td><td>2 h </td></tr><tr><td>None <br></td><td></td></tr><tr><td rowspan="3">E - So many books</td><td>20 iter - 5.031.624</td><td>3 min </td><td rowspan="3">3.275.190<br>pop_size = 100<br>ngen = 400<br>cxpb = 0.5<br>mutpb = 0.7<br>tool_m = 0.02</td><td rowspan="3">circa 2 min per <br>arrivare a <br>2.500.000, <br>converge in <br>circa 10 min  </td><td rowspan="3">5.034.898</td></tr><tr><td>100 iter - 5.064.292</td><td>14 min </td></tr><tr><td>200 iter - 5.083.813</td><td>28 min </td></tr><tr><td rowspan="3">F - Libraries of the <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;world </td><td>20 iter - 5.267.148</td><td>1.5 min </td><td rowspan="3">5.259.020<br>pop_size = 150<br>ngen = 600<br>cxpb = 0.2<br>mutpb = 0.8<br>tool_m = 0.04</td><td rowspan="3">intorno ai 2 min <br>3.500.000, <br>converge in <br>circa 10 min </td><td rowspan="3">5.308.034</td></tr><tr><td>100 iter - 5.328.296</td><td>6 min </td></tr><tr><td>200 iter - 5.332.987</td><td>12 min </td></tr><tr><td>TOTAL SCORE</td><td>26.724.326</td><td>//</td><td>20.580.150</td><td>//</td><td>26.884.761</td><td></td></tr></tbody></table>